# NN tryouts on SPR data, inspired by Kaggle Forum "When less is more"

Load training and validation data as 

    month : [F | T | LC ]


In [1]:
import os
import numpy as np
import pandas as pd

import logging
logging.getLogger().handlers = []
logging.basicConfig(level=logging.INFO)

import matplotlib.pylab as plt
%matplotlib inline

In [9]:
from dataset import load_trainval
from common import to_yearmonth

In [3]:
train_yearmonths_list = [201502, 201503, 201504, 201505, 201602, 201603, 201604]
val_yearmonth = [201605]
train_nb_clients = 250000
train_df, val_df = load_trainval(train_yearmonths_list, val_yearmonth, train_nb_clients)

INFO:root:- Load training data : [201501, 201502, 201503, 201504, 201505, 201601, 201602, 201603, 201604]
INFO:root:-- Select 250000 clients
INFO:root:- Number of lines with unknown data : 145
INFO:root:- Number of columns with nan : 9
INFO:root:- Process date : 201502
/home/osboxes/Documents/TDS/venv/local/lib/python2.7/site-packages/pandas/core/indexing.py:296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/osboxes/Documents/TDS/venv/local/lib/python2.7/site-packages/pandas/core/indexing.py:476: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

Display loaded data

In [4]:
#train_df.head(10)

In [5]:
#mask = (~train_df['diff'].isnull()) & (train_df['diff'] != 0.0)
#train_df[mask].sort_values(['ncodpers', 'fecha_dato'])[['fecha_dato', 'ncodpers', 'targets_str', 'targets_dec', 'diff', 'lc_targets_str', 'lc_targets_dec']].head(10)

In [6]:
#val_df.head(10)

In [7]:
#mask = (~val_df['diff'].isnull()) & (val_df['diff'] != 0.0)
#val_df[mask].sort_values(['ncodpers', 'fecha_dato'])[['fecha_dato', 'ncodpers', 'targets_str', 'targets_dec', 'diff', 'lc_targets_str', 'lc_targets_dec']].head(10)

Check active client performances 2015 -> 2016

In [10]:
months_ym_map = {}
months = train_df['fecha_dato'].unique()
for m in months:
    months_ym_map[to_yearmonth(m)] = m

In [ ]:
yms = [201502, 201503, 201504]

for ym in yms:
    ym1 = ym + 100
    
    mask1 = (~train_df['diff'].isnull()) & (train_df['diff'] != 0.0)
    tmask1 = train_df['fecha_dato'] == months_ym_map[ym]
    active_clients1 = train_df[tmask1 & mask1]['ncodpers'].unique()

    mask2 = (~train_df['diff'].isnull()) & (train_df['diff'] != 0.0)
    tmask2 = train_df['fecha_dato'] == months_ym_map[ym1]
    active_clients2 = train_df[tmask2 & mask2]['ncodpers'].unique()

    active_clients = list(set(active_clients1) & set(active_clients2))   
    
    mask1 &= train_df['ncodpers'].isin(active_clients)
    t1 = train_df[tmask1 & mask1].sort_values(['ncodpers', 'fecha_dato'])[['fecha_dato', 'ncodpers', 'targets_dec', 'diff', 'lc_targets_dec']]

    mask2 &= train_df['ncodpers'].isin(active_clients)
    t2 = train_df[tmask2 & mask2].sort_values(['ncodpers', 'fecha_dato'])[['fecha_dato', 'ncodpers', 'targets_dec', 'diff', 'lc_targets_dec']]

    print "-------"
    print ym, ym1
    total = t1['diff'].shape[0]
    eq = (t1['diff'].values == t2['diff'].values).sum()
    aeq = (np.abs(t1['diff'].values) == np.abs(t2['diff'].values)).sum()
    print "Percentage : eq =", eq*1.0/total, " | aeq =", aeq*1.0/total
    
       

In [23]:
mask1 = (~train_df['diff'].isnull()) & (train_df['diff'] != 0.0)
tmask1 = train_df['fecha_dato'] == '2015-05-28'
train_active_clients = train_df[tmask1 & mask1]['ncodpers'].unique()

mask2 = (~val_df['diff'].isnull()) & (val_df['diff'] != 0.0)
tmask2 = val_df['fecha_dato'] == '2016-05-28'
val_active_clients = val_df[tmask2 & mask2]['ncodpers'].unique()

active_clients = list(set(train_active_clients) & set(val_active_clients))

In [24]:
mask1 &= train_df['ncodpers'].isin(active_clients)
t1 = train_df[tmask1 & mask1].sort_values(['ncodpers', 'fecha_dato'])[['fecha_dato', 'ncodpers', 'targets_dec', 'diff', 'lc_targets_dec']]
t1.head(10)

,fecha_dato,ncodpers,targets_dec,diff,lc_targets_dec
3144382,2015-05-28,15889,2129968,32.0,2129936.0
2933753,2015-05-28,15982,2623521,-6.0,2623527.0
2933858,2015-05-28,16187,623921,-6.0,623927.0
2933864,2015-05-28,16201,2624097,-6.0,2624103.0
2933407,2015-05-28,16641,2689121,-6.0,2689127.0
2933464,2015-05-28,16681,589894,-1.0,589895.0
2933437,2015-05-28,16713,2629159,5120.0,2624039.0
2933394,2015-05-28,16731,2101256,-1.0,2101257.0
2933614,2015-05-28,17059,2097152,-1.0,2097153.0
2933548,2015-05-28,17118,527376,-6.0,527382.0


In [25]:
mask2 &= val_df['ncodpers'].isin(active_clients)
t2 = val_df[tmask2 & mask2].sort_values(['ncodpers', 'fecha_dato'])[['fecha_dato', 'ncodpers', 'targets_dec', 'diff', 'lc_targets_dec']]
t2.head(10)

,fecha_dato,ncodpers,targets_dec,diff,lc_targets_dec
1238761,2016-05-28,15889,2129968,32.0,2129936.0
1238682,2016-05-28,15982,0,-524288.0,524288.0
1238835,2016-05-28,16187,3377,-524290.0,527667.0
1238825,2016-05-28,16201,2624103,38.0,2624065.0
1238575,2016-05-28,16641,2689121,-6.0,2689127.0
1238609,2016-05-28,16681,589894,-1.0,589895.0
1238907,2016-05-28,16713,2629127,-32.0,2629159.0
1239207,2016-05-28,16731,2103297,1.0,2103296.0
1239260,2016-05-28,17059,2097153,1.0,2097152.0
1239284,2016-05-28,17118,526406,6.0,526400.0


In [26]:
total = t1['diff'].shape[0]
eq = (t1['diff'].values == t2['diff'].values).sum()
aeq = (np.abs(t1['diff'].values) == np.abs(t2['diff'].values)).sum()
print "Percentage : eq =", eq*1.0/total, " | aeq =", aeq*1.0/total


Percentage : eq = 0.371883786317  | aeq = 0.563823805061


In [2]:
from common import load_data2, minimal_clean_data_inplace, preprocess_data_inplace, TARGET_LABELS
from common import get_added_products, remove_last_choice, apk, map7_score2

from visualization import visualize_train_test, visualize_folds, compare_two_datasets, compare_folds, compare_folds2

In [3]:
TRAIN_FILE_PATH = os.path.join("..", "data", "train_ver2.csv")
TEST_FILE_PATH = os.path.join("..", "data", "test_ver2.csv")

In [4]:
from common import _to_ym, _to_ym_dec, to_yearmonth

In [5]:
# def load_trainval(train_yearmonths_list, val_yearmonth, train_nb_clients=-1):

train_yearmonths_list = [201505, 201506]
val_yearmonth = [201605]
train_nb_clients = 1000

In [6]:
def _get_prev_ym(ym):
    return _to_ym(_to_ym_dec(ym) - _to_ym_dec(2))

def _ym_list_to_load(ym_list):
    """
    Append previous month to the each yearmonth
    """
    out = []
    for ym in ym_list:
        # get the previous month
        prev_ym = _get_prev_ym(ym)
        if len(out) and out[-1] == prev_ym:
            out += [ym]
        else:
            out += [prev_ym, ym]
    return out

yearmonth_list = _ym_list_to_load(train_yearmonths_list)
print "Load yearmonth_list", yearmonth_list
train_df = load_data2(TRAIN_FILE_PATH, yearmonth_list, train_nb_clients)
minimal_clean_data_inplace(train_df)
preprocess_data_inplace(train_df)

months = train_df['fecha_dato'].unique()
clients = train_df['ncodpers'].unique()
assert len(clients) == (train_df['ncodpers'].value_counts() == len(yearmonth_list)).sum()
ll = len(clients)
months_ym_map = {}
for m in months:
    l = len(train_df[train_df['fecha_dato'] == m]['ncodpers'].unique())
    assert l == ll, "Number of clients should be identical for all monthes. (%s, %s, %s)" % (m, l, ll)
    months_ym_map[to_yearmonth(m)] = m

Load yearmonth_list [201504, 201505, 201506]


INFO:root:-- Select 1000 clients
INFO:root:- Number of lines with unknown data : 0
INFO:root:- Number of columns with nan : 6


In [7]:
print months, months_ym_map

['2015-04-28' '2015-05-28' '2015-06-28'] {201504: '2015-04-28', 201505: '2015-05-28', 201506: '2015-06-28'}


In [9]:
from math import log


LC_TARGET_LABELS = ['lc_' + t for t in TARGET_LABELS]        


def dummies_to_str(row):
    output = ''
    for v in row.values:
        output += str(int(v))
    return output


def dummies_to_decimal(row):
    output = dummies_to_str(row)
    return int(output, 2)

    
def _add_diff_inplace(df, prev_ym_mask, ym_mask):
    """
    df should be imperatively sorted by clients in order to subtract and assign correctly
    """
    tmp_df = df[['fecha_dato','ncodpers']]
    tmp_df.loc[:,'t'] = df[TARGET_LABELS].apply(dummies_to_decimal, axis=1)
    v1 = tmp_df[ym_mask][['ncodpers', 't']]
    v2 = tmp_df[prev_ym_mask][['ncodpers', 't']]
    assert len(v1) == len(v2), "Length of current month and previous month are not equal"
    v2.index = v1.index
    df.loc[ym_mask,'diff'] = v1['t'] - v2['t']
    
    
def _add_clc_inplace(df, prev_ym_mask, ym_mask):
    """
    df should be imperatively sorted by clients in order to subtract and assign correctly
    """
    clients_last_choice = df[prev_ym_mask][['ncodpers'] + TARGET_LABELS]
#     print clients_last_choice[['ncodpers'] + TARGET_LABELS[:1]].head()
    # Prepend "lc_" to column names
    clc_new_cols = clients_last_choice.columns.values
    clc_new_cols[1:] = "lc_" + clc_new_cols[1:]
    
    clients_last_choice.index = df[ym_mask].index
    for c in clc_new_cols[1:]:
        df.loc[ym_mask, c] = clients_last_choice[c]

    df.loc[ym_mask, 'lc_targets_str'] = df[ym_mask][LC_TARGET_LABELS].apply(dummies_to_str, axis=1)
    df.loc[ym_mask, 'lc_targets_dec'] = df[ym_mask][LC_TARGET_LABELS].apply(dummies_to_decimal, axis=1)


# Imperatively sort by clients in order to subtract and assign correctly
train_df2 = train_df.sort_values(['ncodpers', 'fecha_dato'])    

for ym in train_yearmonths_list:
    prev_ym = _get_prev_ym(ym)
    prev_ym_mask = train_df2['fecha_dato'] == months_ym_map[prev_ym]
    ym_mask = train_df2['fecha_dato'] == months_ym_map[ym]

    train_df2.loc[:, 'targets_str'] = train_df2[TARGET_LABELS].apply(dummies_to_str, axis=1)
    train_df2.loc[:, 'targets_dec'] = train_df2[TARGET_LABELS].apply(dummies_to_decimal, axis=1)
    
    _add_diff_inplace(train_df2, prev_ym_mask, ym_mask)
    _add_clc_inplace(train_df2, prev_ym_mask, ym_mask)


mask = (~train_df2['diff'].isnull())
assert (train_df2[mask]['diff'] == train_df2[mask]['targets_dec'] - train_df2[mask]['lc_targets_dec']).all(), "Something is wrong"


In [10]:
mask = (~train_df2['diff'].isnull()) & (train_df2['diff'] != 0.0)
train_df2[mask].sort_values(['ncodpers', 'fecha_dato'])[['fecha_dato', 'ncodpers', 'targets_str', 'targets_dec', 'diff', 'lc_targets_str', 'lc_targets_dec']].head(10)


,fecha_dato,ncodpers,targets_str,targets_dec,diff,lc_targets_str,lc_targets_dec
1678933,2015-06-28,29672,001000010000000000100000,2162720,32.0,001000010000000000000000,2162688.0
1045448,2015-05-28,32861,000010000000110000110000,527408,-2.0,000010000000110000110010,527410.0
1677490,2015-06-28,32861,000010000000110000010011,527379,-29.0,000010000000110000110000,527408.0
1057149,2015-05-28,58908,001000001000100000000000,2131968,-512.0,001000001000101000000000,2132480.0
1689322,2015-06-28,58908,001000001000100000000001,2131969,1.0,001000001000100000000000,2131968.0
1059597,2015-05-28,65267,000010010000100001100001,591969,524282.0,000000010000100001100111,67687.0
1691802,2015-06-28,65267,000010010000100001100111,591975,6.0,000010010000100001100001,591969.0
1031182,2015-05-28,76117,000010001001010001110001,562289,-6.0,000010001001010001110111,562295.0
1034792,2015-05-28,79075,001000000001000000000000,2101248,2097152.0,000000000001000000000000,4096.0
1040500,2015-05-28,97411,000010011001000000000111,626695,-32.0,000010011001000000100111,626727.0


In [14]:
yearmonth_list = _ym_list_to_load(val_yearmonth)
print "Load yearmonth_list", yearmonth_list
val_df = load_data2(TRAIN_FILE_PATH, yearmonth_list, 'max')
minimal_clean_data_inplace(val_df)
preprocess_data_inplace(val_df)

Load yearmonth_list [201604, 201605]


INFO:root:-- Select max clients
INFO:root:- Number of lines with unknown data : 0
INFO:root:- Number of columns with nan : 10


In [17]:
months = val_df['fecha_dato'].unique()
clients = val_df['ncodpers'].unique()
assert len(clients) == (val_df['ncodpers'].value_counts() == len(yearmonth_list)).sum()
ll = len(clients)
months_ym_map = {}
for m in months:
    l = len(val_df[val_df['fecha_dato'] == m]['ncodpers'].unique())
    assert l == ll, "Number of clients should be identical for all monthes. (%s, %s, %s)" % (m, l, ll)
    months_ym_map[to_yearmonth(m)] = m

In [18]:
print months, months_ym_map

['2016-04-28' '2016-05-28'] {201604: '2016-04-28', 201605: '2016-05-28'}


In [19]:
# Imperatively sort by clients in order to subtract and assign correctly
val_df2 = val_df.sort_values(['ncodpers', 'fecha_dato'])    

for ym in val_yearmonth:
    prev_ym = _get_prev_ym(ym)
    prev_ym_mask = val_df2['fecha_dato'] == months_ym_map[prev_ym]
    ym_mask = val_df2['fecha_dato'] == months_ym_map[ym]

    val_df2.loc[:, 'targets_str'] = val_df2[TARGET_LABELS].apply(dummies_to_str, axis=1)
    val_df2.loc[:, 'targets_dec'] = val_df2[TARGET_LABELS].apply(dummies_to_decimal, axis=1)
    
    _add_diff_inplace(val_df2, prev_ym_mask, ym_mask)
    _add_clc_inplace(val_df2, prev_ym_mask, ym_mask)


mask = (~val_df2['diff'].isnull())
assert (val_df2[mask]['diff'] == val_df2[mask]['targets_dec'] - val_df2[mask]['lc_targets_dec']).all(), "Something is wrong"


In [20]:
mask = (~val_df2['diff'].isnull()) & (val_df2['diff'] != 0.0)
val_df2[mask].sort_values(['ncodpers', 'fecha_dato'])[['fecha_dato', 'ncodpers', 'targets_str', 'targets_dec', 'diff', 'lc_targets_str', 'lc_targets_dec']].head(10)


,fecha_dato,ncodpers,targets_str,targets_dec,diff,lc_targets_str,lc_targets_dec
1238761,2016-05-28,15889,001000001000000000110000,2129968,32.0,001000001000000000010000,2129936.0
1238751,2016-05-28,15900,000000010000000001000001,65601,-524288.0,000010010000000001000001,589889.0
1238787,2016-05-28,15925,001000010000000000100000,2162720,-1.0,001000010000000000100001,2162721.0
1238786,2016-05-28,15926,001000010000010000010000,2163728,-1.0,001000010000010000010001,2163729.0
1238785,2016-05-28,15927,000000001000000100000001,33025,-2097152.0,001000001000000100000001,2130177.0
1238783,2016-05-28,15929,001000001000100000110001,2132017,1.0,001000001000100000110000,2132016.0
1238769,2016-05-28,15944,001000011000000001000001,2195521,-32.0,001000011000000001100001,2195553.0
1238791,2016-05-28,15950,000000000000000101000001,321,-2097152.0,001000000000000101000001,2097473.0
1238734,2016-05-28,15952,001000000000000001000001,2097217,1.0,001000000000000001000000,2097216.0
1238682,2016-05-28,15982,000000000000000000000000,0,-524288.0,000010000000000000000000,524288.0


In [27]:
mask = val_df2['fecha_dato'] == months_ym_map[201605]
val_df2[mask].shape

(926663, 75)

In [22]:
yearmonth_list

[201604, 201605]

In [25]:
full_val_df = load_data2(TRAIN_FILE_PATH, [201605])
minimal_clean_data_inplace(full_val_df)
preprocess_data_inplace(full_val_df)

INFO:root:- Number of lines with unknown data : 0
INFO:root:- Number of columns with nan : 10


In [26]:
full_val_df.shape

(931453, 46)

Load data + minimal cleaning + preprocessing

* 201505 - to get the clients last choice 
* 201506 - to train on

In [ ]:
yearmonth_list = [201504, 201505] 
nb_months = len(yearmonth_list)
nb_clients = 250000
# nb_clients = 'max'

In [ ]:
data_df = load_data2(TRAIN_FILE_PATH, yearmonth_list, nb_clients)
minimal_clean_data_inplace(data_df)
preprocess_data_inplace(data_df)

months = data_df['fecha_dato'].unique()
clients = data_df['ncodpers'].unique()
assert len(clients) == (data_df['ncodpers'].value_counts() == nb_months).sum()
ll = len(clients)
for m in months:
    l = len(data_df[data_df['fecha_dato'] == m]['ncodpers'].unique())
    assert l == ll, "Number of clients should be identical for all monthes. (%s, %s, %s)" % (m, l, ll)

In [ ]:
features = [
    u'ind_empleado', u'pais_residencia',
    u'sexo', u'age', u'ind_nuevo', u'antiguedad', u'indrel',
    u'ult_fec_cli_1t', u'indrel_1mes', u'tiprel_1mes', u'indresi',
    u'indext', u'conyuemp', u'canal_entrada', u'indfall', u'nomprov',
    u'ind_actividad_cliente', u'renta', u'segmento'    
]

In [ ]:
from common import PREPROCESS_LABEL_ENCODERS
#PREPROCESS_LABEL_ENCODERS

In [ ]:
print months

for ym in yearmonth_list:
    print ym

Create profiles and create models for profiles

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Merge
from keras.utils import np_utils

## Select only clients that choose new products in 201506 comparing with 201505

201505 [FEATURES|TARGETS]  
201506 [FEATURES|TARGETS]  

TARGETS_201505 -> clients_last_choice
TARGETS_201506 - TARGETS_201505 -> active_clients

Cross-validation : 
FEATURES_201506 -> (Split train/val) :
  --> (0.75*FEATURES) -> (select only active clients) -> X_train
  --> 0.25*FEATURES -> X_val



In [ ]:
trainval_df = data_df.sort_values(['fecha_dato', 'ncodpers'])
last_choice_mask = trainval_df['fecha_dato'] == months[-2]
train_month_mask = trainval_df['fecha_dato'] == months[-1]

In [ ]:
def add_diff_inplace(df, last_choice_mask, train_month_mask):
    tmp_df = df[['fecha_dato','ncodpers']]
    tmp_df.loc[:,'target'] = df[TARGET_LABELS].sum(axis=1)
    v1 = tmp_df[train_month_mask]['target'].values
    v2 = tmp_df[last_choice_mask]['target'].values
    ll = min(len(v1), len(v2))
    indices = tmp_df.index[ll:]
    df.loc[indices,'diff'] = pd.Series(v1 - v2, index=indices)
    del tmp_df, v1, v2

In [ ]:
add_diff_inplace(trainval_df, last_choice_mask, train_month_mask)
print trainval_df.shape

In [ ]:
clients_last_choice = trainval_df[last_choice_mask][['ncodpers'] + TARGET_LABELS].sort_values(['ncodpers'])
X = trainval_df[train_month_mask][['ncodpers', 'diff'] + features]
Y = trainval_df[train_month_mask][['ncodpers'] + TARGET_LABELS]

In [ ]:
# Select only clients from test data
clients_last_choice = clients_last_choice[clients_last_choice['ncodpers'].isin(X['ncodpers'])]
assert (X['ncodpers'].values == clients_last_choice['ncodpers'].values).all(), "WTF"

### Feature engineering

In [ ]:
def get_age_group_index(age):
    if age < 10:
        return -3
    elif age < 15:
        return -2        
    elif age < 18:
        return -1    
    elif age < 23:
        return 0
    elif age < 25:
        return 1
    elif age < 27:
        return 2
    elif age < 28:
        return 3    
    elif age < 32:
        return 4
    elif age < 37:
        return 5    
    elif age < 42:
        return 6
    elif age < 47:
        return 7
    elif age < 52:
        return 8
    elif age < 57:
        return 9
    elif age < 60:
        return 10
    elif age < 65:
        return 11
    elif age < 70:
        return 12
    elif age < 75:
        return 13
    elif age < 80:
        return 14
    else:
        return 15

def get_income_group_index(income):
    if income < 0:
        return -1
    elif income < 45542.97:
        return 1
    elif income < 57629.67:
        return 2
    elif income < 68211.78:
        return 3
    elif income < 78852.39:
        return 4
    elif income < 90461.97:
        return 5
    elif income < 103855.23:
        return 6
    elif income < 120063.00:
        return 7
    elif income < 141347.49:
        return 8
    elif income < 173418.36:
        return 9
    elif income < 234687.12:
        return 10
    else:
        return 11

In [ ]:
X.loc[:,'age'] = X['age'].apply(get_age_group_index)
X.loc[:,'renta'] = X['renta'].apply(get_income_group_index)

In [ ]:
# X_countries = pd.get_dummies(X['pais_residencia'])
# X_countries.head()

In [ ]:
# def drop_const_cols(df):
#     """
#     Method to remove constant columns
#     """
#     indices = df.index
#     return df.loc[:, (df != df.ix[indices[0]]).any()]

# X = drop_const_cols(X)

In [ ]:
# X_dummy = pd.DataFrame()

# X_dummy = pd.concat([X_dummy, X['diff']], axis=1)
# ff = list(features)
# ff.remove('indrel_1mes')
# ff.remove('tiprel_1mes')
# ff = list(set(ff) & set(X.columns))

# for f in ff:
#     print "Process dummification: ", f
#     X_dummy = pd.concat([X_dummy, pd.get_dummies(X[f])], axis=1)
    
# print X_dummy.shape
# X_dummy.head()

In [ ]:
X.head()

### Run KFold Cross-validation 

In [ ]:
def create_model(input_dim, output_dim):
        
    model = Sequential()
    model.add(Dense(input_dim, init='uniform', input_dim=input_dim, activation='relu'))
    model.add(Dropout(0.30))
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.20))
    model.add(Dense(30, activation='relu'))
    model.add(Dropout(0.10))
    model.add(Dense(output_dim, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
    return model

In [ ]:
from sklearn.model_selection import KFold

nb_folds=3

def max_map7_score_per_fold(x_df, y_df, clients_last_choice_df, nb_folds):    
    kf = KFold(n_splits=nb_folds)
    scores = []
    for train_index, test_index in kf.split(range(x_df.shape[0])):
        X_val = x_df.loc[x_df.index[test_index], :]
        Y_val = y_df.loc[y_df.index[test_index], :]
        CLC_val = clients_last_choice_df.loc[clients_last_choice_df.index[test_index], :]                
        x_val, y_val = prepare_to_test(X_val, Y_val)
        scores.append(map7_score2(y_val, y_val, CLC_val[TARGET_LABELS].values))  
    return scores        

In [ ]:
# CROSS VALIDATION
from sklearn.preprocessing import StandardScaler
n_highest = 7

def prepare_to_fit(X_train, Y_train):
    # Select only active client for the training part
    mask = X_train['diff'] > 0
    x_train = X_train[mask].drop(['diff'], axis=1).values
    y_train = Y_train[mask][TARGET_LABELS].values
    x_train = StandardScaler().fit_transform(x_train)
    return x_train, y_train

def prepare_to_fit_nomask(X_train, Y_train):
    y_train = Y_train[TARGET_LABELS].values
    x_train = X_train.drop(['diff'], axis=1).values
    x_train = StandardScaler().fit_transform(x_train)
    return x_train, y_train

def prepare_to_test(X_val, Y_val=None):
    if Y_val is not None:
        y_val = Y_val[TARGET_LABELS].values
    
    x_val = []
    x_val = X_val.drop(['diff'], axis=1).values    
    x_val = StandardScaler().fit_transform(x_val)  
    
    if Y_val is not None:
        return x_val, y_val
    else:
        return x_val

def cross_val_score2(data, 
                     nb_folds=5, 
                     prepare_to_fit_func=prepare_to_fit, 
                     prepare_to_test_func=prepare_to_test, 
                     create_model_func=create_model):
    
    x_df, y_df, clients_last_choice_df = data
    kf = KFold(n_splits=nb_folds)
    scores = []
    
    for train_index, test_index in kf.split(range(x_df.shape[0])):
        X_train, X_val = x_df.loc[x_df.index[train_index], :], x_df.loc[x_df.index[test_index], :]
        Y_train, Y_val = y_df.loc[y_df.index[train_index], :], y_df.loc[y_df.index[test_index], :]
        CLC_val = clients_last_choice_df.loc[clients_last_choice_df.index[test_index], :]
                
        x_train, y_train = prepare_to_fit_func(X_train, Y_train)
        x_val, y_val = prepare_to_test_func(X_val, Y_val)
        
        logging.info("- Train/Val shapes : {}, {} | {}, {}".format(
                [i.shape for i in x_train] if isinstance(x_train, list) else x_train.shape, 
                [i.shape for i in x_val] if isinstance(x_val, list) else x_val.shape, 
                y_train.shape, 
                y_val.shape)
        )                
            
        logging.info("- Create the model : ")
        estimator = create_model_func(x_train.shape[1], len(TARGET_LABELS))
        logging.info("- Fit the model")
        hist = estimator.fit(x_train, y_train, nb_epoch=250, batch_size=5000, verbose=0)        
        for key in hist.history:            
            logging.info("-- %s : min=%f, max=%f" % (key, np.min(hist.history[key]), np.max(hist.history[key])))
        
        logging.info("- Predict using trained model")
        y_pred = estimator.predict(x_val, verbose=0)
        logging.info("- Compute map7 score")
        scores.append(map7_score2(y_val, y_pred, CLC_val[TARGET_LABELS].values))        
    
    return np.array(scores)

In [ ]:
results = cross_val_score2((X.drop(['ncodpers'], axis=1), Y, clients_last_choice), 
                           nb_folds=nb_folds)
print "Cross-Validation \n %i | %f | %f | %f | %.5f " % (nb_folds, results.min(), results.mean(), results.max(), results.std())

### Test 1 : Train on active clients + feature engineering

#### Data: 

- yearmonth_list = [201505, 201506]
- nb_clients = 250000

#### Feature engineering

- age -> age group
- income -> income group


#### Model

- 75 -> 50 -> 30 ->
- sigmoid, binary_crossentropy, nadam, accuracy, 

**Conf:**
- batch_size=2000

*Cross-validation results :* 

 Nb epoch | Nb folds | Min MAP@7 | Mean MAP@7 | Max MAP@7 | STD MAP@7
 --- | --- | --- | --- | --- | ---
 150 | 3 | 0.025659 | 0.029959 | 0.037639 | 0.00544 


#### Model 1 

- 50 -> 30 ->
- sigmoid, binary_crossentropy, nadam, accuracy, 

**Conf:**
- batch_size=2000

*Cross-validation results :* 

 Nb epoch | Nb folds | Min MAP@7 | Mean MAP@7 | Max MAP@7 | STD MAP@7
 --- | --- | --- | --- | --- | ---
 150 | 3 | 0.025802 | 0.030137 | 0.037217 | 0.00505 


Kaggle : 0.0197579

## Train model for predictions

In [ ]:
def train_model(X_train, Y_train):

    x_train, y_train = prepare_to_fit(X_train, Y_train)
    logging.info("- Train data shapes : {}, {}".format(
            [i.shape for i in x_train] if isinstance(x_train, list) else x_train.shape, 
            y_train.shape)
    )                

    logging.info("- Create the model")
    
    estimator = create_model(x_train.shape[1], len(TARGET_LABELS))
    logging.info("- Fit the model")
    hist = estimator.fit(x_train, y_train, nb_epoch=150, batch_size=2000, verbose=0)        
    for key in hist.history:            
        logging.info("-- %s : min=%f, max=%f" % (key, np.min(hist.history[key]), np.max(hist.history[key])))
    return estimator

estimator = train_model(X[['diff'] + features], Y)

Check score on the data 2016

In [ ]:
yearmonth_list = [201604, 201605] 
nb_months = len(yearmonth_list)
nb_clients = 'max'

In [ ]:
val_df = load_data2(TRAIN_FILE_PATH, yearmonth_list, nb_clients)
minimal_clean_data_inplace(val_df)
preprocess_data_inplace(val_df)

months = val_df['fecha_dato'].unique()
clients = val_df['ncodpers'].unique()
assert len(clients) == (val_df['ncodpers'].value_counts() == nb_months).sum()
ll = len(clients)
for m in months:
    l = len(val_df[val_df['fecha_dato'] == m]['ncodpers'].unique())
    assert l == ll, "Number of clients should be identical for all monthes. (%s, %s, %s)" % (m, l, ll)

In [ ]:
val_df = val_df.sort_values(['fecha_dato', 'ncodpers'])
last_choice_mask = val_df['fecha_dato'] == months[-2]
train_month_mask = trainval_df['fecha_dato'] == months[-1]

In [ ]:
def add_diff_inplace(df, last_choice_mask, train_month_mask):
    tmp_df = df[['fecha_dato','ncodpers']]
    tmp_df.loc[:,'target'] = df[TARGET_LABELS].sum(axis=1)
    v1 = tmp_df[train_month_mask]['target'].values
    v2 = tmp_df[last_choice_mask]['target'].values
    ll = min(len(v1), len(v2))
    indices = tmp_df.index[ll:]
    df.loc[indices,'diff'] = pd.Series(v1 - v2, index=indices)
    del tmp_df, v1, v2

In [ ]:
add_diff_inplace(trainval_df, last_choice_mask, train_month_mask)
print trainval_df.shape

In [ ]:
clients_last_choice = trainval_df[last_choice_mask][['ncodpers'] + TARGET_LABELS].sort_values(['ncodpers'])
X = trainval_df[train_month_mask][['ncodpers', 'diff'] + features]
Y = trainval_df[train_month_mask][['ncodpers'] + TARGET_LABELS]

In [ ]:
# Select only clients from test data
clients_last_choice = clients_last_choice[clients_last_choice['ncodpers'].isin(X['ncodpers'])]
assert (X['ncodpers'].values == clients_last_choice['ncodpers'].values).all(), "WTF"

## Prediction on test data

Load the last month from the training dataset to get user last choice and remove it from predictions

In [ ]:
yearmonth_list = [201605]
lastmonth_df = load_data2(TRAIN_FILE_PATH, yearmonth_list)
minimal_clean_data_inplace(lastmonth_df)

test_df = load_data2(TEST_FILE_PATH, [])
minimal_clean_data_inplace(test_df)
preprocess_data_inplace(test_df)
test_df = test_df.sort_values(['ncodpers'])

clients_last_choice_test = lastmonth_df[['ncodpers'] + TARGET_LABELS].sort_values(['ncodpers'])
# Select only clients from test data
clients_last_choice_test = clients_last_choice_test[clients_last_choice_test['ncodpers'].isin(test_df['ncodpers'])]
assert (test_df['ncodpers'].values == clients_last_choice_test['ncodpers'].values).all(), "WTF"

In [ ]:
X_test = test_df[['ncodpers'] + features]

In [ ]:
X_test.loc[:,'age'] = X_test['age'].apply(get_age_group_index)
X_test.loc[:,'renta'] = X_test['renta'].apply(get_income_group_index)

Make predictions

In [ ]:
def get_submission(y_pred, clients, clc, target_labels, n_highest=7):
    predicted_added_products = np.argsort(y_pred, axis=1)
    predicted_added_products = predicted_added_products[:,::-1][:,:n_highest]
    added_products_col = []
    count = 0 
    for products, last_choice in zip(predicted_added_products, clc):
        predictions = remove_last_choice(products, last_choice)
        added_products_col.append(' '.join([target_labels[i] for i in predictions]))
        count+=1
        if count % 100000 == 0:
            logging.info("Elapsed : %i", count)
            
    out = pd.DataFrame(data={'ncodpers': clients, 'added_products': added_products_col}, columns=['ncodpers', 'added_products'])
    return out

In [ ]:

data = {
    0: {'x': X_test, 'clc': clients_last_choice_test},
}

submissions = []
for i in data.keys():
    x = data[i]['x']
    clc = data[i]['clc']
    x_test = prepare_to_test(x)
    y_pred = estimator.predict(x_test, verbose=0)

    clients = x['ncodpers']
    submission = get_submission(y_pred, clients, clc[TARGET_LABELS].values, TARGET_LABELS, 7)
    print submission.head()
    submissions.append(submission)

submission = pd.concat(submissions)

In [ ]:
submission.head()

Get submission DataFrame and write csv file

In [ ]:
from datetime import datetime
import csv

logging.info('- Generate submission')
submission_file = '../results/submission_' + \
                  str(datetime.now().strftime("%Y-%m-%d-%H-%M")) + \
                  '.csv'

submission.to_csv(submission_file, index=False, index_label=False)